In [30]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [31]:
cap = cv2.VideoCapture('Robots.mp4')

In [32]:
colors = [ np.random.randint(0, 255, size=(3, )) for i in range(200)]
colors = [ ( int(color[0]), int(color[1]), int(color[2]) ) for color in colors]

def capture_moving_sparse(prev_frame, cur_frame, feat1, mask):
    b,g,r = cv2.split(prev_frame) # Changing the order from bgr to rgb so that matplotlib can show it
    img1 = cv2.merge([r,g,b])
    b,g,r = cv2.split(cur_frame) # Changing the order from bgr to rgb so that matplotlib can show it
    img2 = cv2.merge([r,g,b])

    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    feat2, status, error = cv2.calcOpticalFlowPyrLK(gray1, gray2, feat1, None)
    
    for i, (feature1, feature2) in enumerate(zip(feat1, feat2)):
        f10=int(feature1[0][0])
        f11=int(feature1[0][1])
        f20=int(feature2[0][0])
        f21=int(feature2[0][1])

        # Choose treshold
        if(((f10-f20)**2+(f11-f21)**2)**(1/2)) > 1.5:
            cv2.circle(img2, (f10, f11), 5, (0, 255, 0), -1)
            cv2.line(mask, (f10,f11), (f20, f21), tuple(colors[i]), 5)
        
    img2 = cv2.add(img2, mask)
    return img2, feat2

In [33]:

ret, prev_frame = cap.read()
line = []

b,g,r = cv2.split(prev_frame) # Changing the order from bgr to rgb so that matplotlib can show it
img1 = cv2.merge([r,g,b])
gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
features = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.2, minDistance=5)
mask = np.zeros_like(prev_frame)

while ret:
    ret, cur_frame = cap.read()
    outcome, features = capture_moving_sparse(prev_frame, cur_frame, features, mask)
    resized = cv2.resize(outcome, (960, 540))
    cv2.imshow("frame", resized)
    prev_frame = cur_frame
    if cv2.waitKey(5) & 0xFF == ord('q'): break
cap.release()


In [34]:
cv2.destroyAllWindows()

# Second part

In [45]:
import math
def capture_moving_dense(prev_frame, cur_frame):
    b,g,r = cv2.split(prev_frame) # Changing the order from bgr to rgb so that matplotlib can show it
    img1 = cv2.merge([r,g,b])
    b,g,r = cv2.split(cur_frame) # Changing the order from bgr to rgb so that matplotlib can show it
    img2 = cv2.merge([r,g,b])

    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.5, 0)
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1]) # Retrieving the magnitude and angle of every pixel
    for row, (mag_rows,ang_rows) in enumerate(zip(mag,ang)):
        for column, (magnitude, angle) in enumerate(zip(mag_rows, ang_rows)):
            if row % 10 == 0 and column % 10 == 0:
                magnitude *= 10
                arg = (column, row)
                if magnitude == float("infinity"): continue
                cv2.line(img2, (arg), (arg[0] + int(math.cos(angle)*magnitude), arg[1]+int(math.sin(angle)*magnitude)), (0, 0, 255), 1)
                cv2.circle(img2, (arg), 2, (255, 0, 0), -1)
    return img2

In [46]:
cap = cv2.VideoCapture('Robots.mp4')
ret, prev_frame = cap.read()
line = []

b,g,r = cv2.split(prev_frame) # Changing the order from bgr to rgb so that matplotlib can show it
img1 = cv2.merge([r,g,b])
gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
features = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.2, minDistance=5)
mask = np.zeros_like(prev_frame)

while ret:
    ret, cur_frame = cap.read()
    outcome = capture_moving_dense(prev_frame, cur_frame)
    resized = cv2.resize(outcome, (960, 540))
    cv2.imshow("frame", resized)
    prev_frame = cur_frame
    if cv2.waitKey(5) & 0xFF == ord('q'): break
cap.release()

In [47]:
cv2.destroyAllWindows()